In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
from os.path import join as path
from torchvision.transforms import (
    Compose, Resize, Normalize, Grayscale, 
    ToPILImage, ToTensor, RandomHorizontalFlip)
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


tform = {'train': Compose([
                           Resize((128, 128)),
                           ToTensor(),
                           RandomHorizontalFlip(),
                           Normalize((0.485, 0.456, 0.406),
                                     (0.229, 0.224, 0.225))]),
         'valid': Compose([
                           Resize((128, 128)),
                           ToTensor(),
                           Normalize((0.485, 0.456, 0.406),
                                     (0.229, 0.224, 0.225))]),
         'test': Compose([
                           Resize((128, 128)),
                           ToTensor(),
                           Normalize((0.485, 0.456, 0.406),
                                     (0.229, 0.224, 0.225))])}

data_dir = '/content/drive/MyDrive/Colab Notebooks/dogImages'
dataset = {x: ImageFolder(path(data_dir, x), tform[x]) for x in tform.keys()}
loaders = {x: DataLoader(dataset=dataset[x],
                         batch_size=32,
                         shuffle=(x=='train'),
                        #  num_workers=4,
                         pin_memory=True,
                         drop_last=True) for x in tform.keys()}

In [4]:
from torch.nn import Module, Conv2d, MaxPool2d, Linear
from torch.nn.functional import relu, softmax, dropout
from torch import cuda, device

class Model(Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3))
        self.conv2 = Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3))
        self.pool1 = MaxPool2d(kernel_size=(2,2))

        self.conv3 = Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3))
        self.conv4 = Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3))
        self.pool2 = MaxPool2d(kernel_size=(2,2))

        self.conv5 = Conv2d(in_channels=512, out_channels=1024, kernel_size=(3,3))
        self.conv6 = Conv2d(in_channels=1024, out_channels=2048, kernel_size=(3,3))
        self.pool3 = MaxPool2d(kernel_size=(2,2))

        self.fc1 = Linear(12*12*2048, 512)
        self.fc2 = Linear(512, 133)


    def forward(self, x, show_shape=False):
        if show_shape: print(x.squeeze(0).shape)
        x = relu(self.conv1(x))
        if show_shape: print(x.squeeze(0).shape)
        x = relu(self.conv2(x))
        if show_shape: print(x.squeeze(0).shape)
        x = self.pool1(x)
        if show_shape: print(x.squeeze(0).shape)
        x = relu(self.conv3(x)) 
        if show_shape: print(x.squeeze(0).shape)
        x = relu(self.conv4(x))
        if show_shape: print(x.squeeze(0).shape)
        x = self.pool2(x)
        if show_shape: print(x.squeeze(0).shape)
        x = relu(self.conv5(x)) 
        if show_shape: print(x.squeeze(0).shape)
        x = relu(self.conv6(x))
        if show_shape: print(x.squeeze(0).shape)
        x = self.pool3(x)
        if show_shape: print(x.squeeze(0).shape)

        x = x.view(x.size(0), -1)
        if show_shape: print(x.squeeze(0).shape)
        x = self.fc1(x)
        if show_shape: print(x.squeeze(0).shape)
        x = self.fc2(x)
        if show_shape: print(x.squeeze(0).shape)

        return x

model = Model()

use_cuda = cuda.is_available()
if use_cuda:
    model = model.cuda()

In [5]:
test_shape = dataset['train'][0][0].unsqueeze(0)
if use_cuda:
    test_shape = test_shape.cuda()
model(test_shape, show_shape=True);

torch.Size([3, 128, 128])
torch.Size([64, 126, 126])
torch.Size([128, 124, 124])
torch.Size([128, 62, 62])
torch.Size([256, 60, 60])
torch.Size([512, 58, 58])
torch.Size([512, 29, 29])
torch.Size([1024, 27, 27])
torch.Size([2048, 25, 25])
torch.Size([2048, 12, 12])
torch.Size([294912])
torch.Size([512])
torch.Size([133])


In [6]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from torch import save

epochs = 10

def train(model, loaders, criterion, optimizer, epochs):
    min_loss = 2**64
    for epoch in range(epochs):
        print(f'epoch: {epoch+1}')

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            elif phase == 'valid':
                model.eval()

            epoch_loss = 0.0
            for data, cls in loaders[phase]:
                if use_cuda:
                    data = data.cuda()
                    cls = cls.cuda()
                
                if phase == 'train':
                    optimizer.zero_grad()

                loss = criterion(model(data), cls)
                
                epoch_loss += loss.item() * data.size(0)
                print(loss.item())

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            epoch_loss /= len(loaders[phase].dataset)

            print(f'epoch {epoch+1}: {phase} phase is completed. mean loss: {epoch_loss}')

        if min_loss > epoch_loss:
            save(model.state_dict(), 'model.pt')
            min_loss = epoch_loss
            print(f'minimum loss: {min_loss}. Model saved')

train(model, loaders, criterion, optimizer, epochs)

In [8]:
def test(model, loaders, criterion):
    model.eval()
    for data, cls in loaders['test']:
        if use_cuda:
            data = data.cuda()
            cls = cls.cuda()

        pred = model(data)
        loss = criterion(pred, cls)